In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Uni/Deep Learning for Computer Vision/GeoGuessr_Project')

# folder with pictures
ROOT_DIR = r'C:\Users\Shadow\Pictures\Geogussr\Projekt'

# dir to csv files
dir = r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\preprocess"

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
#from torchsummary import summary
from torch.optim import lr_scheduler


# Dataset
from GeoGuessrDataset import GeoGuessrDataset


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tnrange
import time
import copy
import pygeohash as phg


%matplotlib inline

# Enable autoreloading of imported modules.
%load_ext autoreload
%autoreload 2

In [3]:

# Check GPU support on your machine.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

HEIGHT = 512
WIDTH = 2560

cuda:0


In [4]:

# from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.resnet import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights


# New weights with accuracy 80.858%
resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
resnet50(weights=ResNet50_Weights.DEFAULT)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:



df = pd.read_csv(dir+"\coordinates.csv", delimiter=',', skiprows=0, low_memory=False)

# We want a geohash precsion of 3 so that we get approximately 32768 cells, which will represent our classes.
df['geohash']=df.apply(lambda coords: phg.encode(coords.latitude, coords.longitude, precision=3), axis=1)


def geohash_to_decimal(geohash):
    base_32 = '0123456789bcdefghjkmnpqrstuvwxyz';
    geohash = geohash.lower()
    # Check that the input geohash is a valid geohash
    if not all(c in base_32 for c in geohash):
        raise ValueError('Invalid geohash')
    return sum([32**idx * base_32.index(char) for idx, char in enumerate(geohash[::-1])])

df['geohash_decimal']=df.apply(lambda x: geohash_to_decimal(x["geohash"]) ,axis=1)

geohashes_with_samples = df["geohash_decimal"].unique()
print("Number of geohashes with samples", len(geohashes_with_samples))

geohash_map = { geo: i for i, geo in enumerate(geohashes_with_samples)}

df["geo_code"] = df.apply(lambda geohash: geohash_map[geohash["geohash_decimal"]], axis=1)

df[["filename", "latitude","longitude", "geohash_decimal", "geo_code"]].to_csv(dir+"\coordinates2.csv", index=False)

# Add geohash center coordinates to csv

def geohash_center(geohash_name):

    # Decode the geohash to get the center coordinates and errors.
    latitude, longitude, latitude_error, longitude_error = phg.decode_exactly(geohash_name)
    return [latitude, longitude]

df["geo_lat"], df["geo_lon"] = df.apply(lambda x: geohash_center(x["geohash"])[0] ,axis=1), df.apply(lambda x: geohash_center(x["geohash"])[1] ,axis=1)

# drop the duplicates
df = df.drop(columns=["filename", "latitude", "longitude","geohash_decimal","geohash"] )
df = df.drop_duplicates()
array = df.to_numpy()
array = np.array(array, dtype=np.float64)
tensor = torch.tensor(array)
torch.save(tensor, dir+'\\tensor.pt')
# Save the DataFrame to a CSV file.
df[["geo_code", "geo_lat", "geo_lon"]].to_csv(dir+"\coords_center.csv", index=False)


Number of geohashes with samples 3139


In [6]:
# Define the data transformation
transform = transforms.Compose([
    transforms.ToTensor(),  # convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # normalize images
    transforms.Resize((250, 1000))
])

# Load the dataset and split it into training and validation sets
dataset = GeoGuessrDataset(csv_file=dir+'\coordinates2.csv',
                                    root_dir=ROOT_DIR, transform=transform, num_classes=3139)
train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
print(len(train_dataset))
# Define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)

dataloaders = {"train" : train_dataloader, "val": val_dataloader}
dataset_sizes = {"train": train_size, "val" : val_size}


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



for i, batch in enumerate(train_dataloader):
    x, y, z = batch["image"], batch["geohash"], batch['gt']
    print(x.shape, y.shape, z.shape)
    break
    

127474
torch.Size([64, 3, 250, 1000]) torch.Size([64, 3139]) torch.Size([64, 2])


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    torch.backends.cudnn.benchmark = True
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    batch=next(iter(dataloaders["train"]))
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #for idx, batch in enumerate(dataloaders[phase]):
            inputs, labels, gt = batch["image"], batch["geohash"], batch['gt']
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
                print(criterion2(outputs, gt))
                loss = criterion(outputs, labels.float())

                ######
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    
                    optimizer.step()
            _,labels = torch.max(labels, 1)
            # statistics
            running_loss += loss.item() * inputs.size(0)
            #print(preds)
            #print(labels)
            running_corrects += torch.sum(preds == labels)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / 64 #dataset_sizes[phase]
            epoch_acc = running_corrects.double() / 64 #dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
import math
def haversine_loss(lons1, lats1, lons2, lats2):
    """
    Calculate the Haversine distance between two sets of longitudes and latitudes.

    Parameters:
        - lons1 (torch.tensor): Tensor of longitudes for the first set of points.
        - lats1 (torch.tensor): Tensor of latitudes for the first set of points.
        - lons2 (torch.tensor): Tensor of longitudes for the second set of points.
        - lats2 (torch.tensor): Tensor of latitudes

    Returns:
        - distances (torch.tensor): Tensor of Haversine distances.
    """
    # Convert longitudes and latitudes to radians.
    lons1 = lons1.to(torch.float) * math.pi / 180
    lats1 = lats1.to(torch.float) * math.pi / 180
    lons2 = lons2.to(torch.float) * math.pi / 180
    lats2 = lats2.to(torch.float) * math.pi / 180

    # Calculate differences in longitudes and latitudes.
    dlons = lons2 - lons1
    dlats = lats2 - lats1

    # Calculate intermediate values.
    a = (torch.sin(dlats / 2)**2) + (torch.cos(lats1) * torch.cos(lats2) * (torch.sin(dlons / 2)**2))
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    # Calculate distances.
    R = 6371  # Earth radius in kilometers
    distances = R * c

    return distances

In [9]:

# custom loss that transforms the predicted cluster and the ground truth label into coordinates
# and then computes the haversine distance between them as loss


class HaversineLoss(torch.nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()


    def forward(self, output, gt):
        """
            output: (batchsize, num_clusters) Tensor with probabillities
            gt: tupel contains groundtruth lat, lon
        """

        
        tensor = torch.load(dir+'\\tensor.pt')
        # Extract the latitude and longitude from the predicted location.
        #lat_cluster, lon_cluster = torch.zeros(len(output)), torch.zeros(len(output))
        lat_cluster, lon_cluster = tensor[:,1], tensor[:,2]

        # Calculate the Haversine distance between the predicted and target locations.
        gt0 = torch.tensor(gt[:,0])
        gt1 = torch.tensor(gt[:,1])
        
        distance = haversine_loss(lat_cluster, lon_cluster, gt0[:,None], gt1[:,None])
        #print(torch.argmin(distance,dim=1))    
        
        
        output = F.softmax(output, dim=1)
        
        # Return the loss.
        distance = distance.to(device)
        print(torch.sum(distance*output,dim=1))
        
        loss = torch.mean(torch.sum(distance*output,dim=1))

        return loss


In [10]:
model_ft = models.resnet18(weights=ResNet18_Weights.DEFAULT)
# model_ft = models.resnet18()

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 3139)


model_ft = model_ft.to(device)

criterion = nn.MSELoss()
 
criterion2 = HaversineLoss()
# Observe that all parameters are being optimized
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=200, gamma=0.1)


In [11]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------


C:\Users\Shadow\AppData\Local\Temp\ipykernel_18340\3723925526.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt0 = torch.tensor(gt[:,0])
C:\Users\Shadow\AppData\Local\Temp\ipykernel_18340\3723925526.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt1 = torch.tensor(gt[:,1])


tensor([ 8317.4912,  9860.8545,  9227.1504,  9871.3789, 10257.8867,  9573.2607,
         9469.5908,  9598.7998,  9193.7012, 10782.8867,  9328.4346,  8331.4355,
         9217.6768,  8348.5127, 11070.0742,  8925.2998,  8348.4102,  8326.8252,
        11138.6455, 10797.1172,  8393.9316, 11179.7344,  8342.4766,  9508.3477,
         9135.6719,  8289.0859, 10986.9580, 10852.9805,  8394.6875, 10774.4688,
         8513.6035,  9290.9180,  9340.1748, 10476.6416,  8767.4658,  9295.7988,
        10970.9131, 10827.6953,  8324.3721,  9000.5586, 10725.6953,  8293.3779,
         9262.8828,  8657.7793, 10848.9785,  8348.4375, 11050.3262,  9301.9580,
         9211.8428, 10449.2207,  9181.8633,  9675.6543, 10520.4590,  8347.9336,
         8692.1680,  9719.9336,  9292.6621, 11178.3975,  8318.8174,  9255.6455,
         9154.0967,  9178.7773,  8913.3818,  9255.4316], device='cuda:0',
       grad_fn=<SumBackward1>)
tensor(9491.9941, device='cuda:0', grad_fn=<MeanBackward0>)
train Loss: 0.3302 Acc: 0.0000
tens

train Loss: 0.0361 Acc: 0.0000
tensor([ 7994.0679,  9543.4453,  8847.4268,  9601.6035, 10789.4033,  9204.6953,
         9108.9297,  9216.2432,  9031.2988, 11631.5918,  9060.8047,  7642.4971,
         9051.2939,  7948.1748, 11418.9082,  8943.6396,  7433.8193,  7779.6475,
        11339.2979, 11139.3652,  8181.4990, 12202.7715,  7793.4497,  9199.0801,
         8595.6797,  7860.5864, 11317.6240, 11363.8770,  8214.3984, 11070.9795,
         7959.1973,  8758.4102,  8921.7793, 10414.5762,  8834.5918,  8874.9004,
        11230.1543, 11517.7100,  7723.1533,  8795.7520, 11819.5215,  7762.6396,
         9031.2539,  8634.9121, 11055.3945,  8013.7129, 11383.9199,  9090.9473,
         8885.6426, 10297.2031,  8733.5166,  9424.8027, 10535.3691,  7255.2129,
         8561.5547,  9465.9746,  8682.1836, 11806.3945,  7426.3950,  8896.9707,
         8701.8125,  8812.7715,  8774.9219,  9059.8770], device='cuda:0')
tensor(9338.5820, device='cuda:0')
val Loss: 14.7996 Acc: 0.0000

Training complete in 0m 37s
B

In [12]:
"""
def imshow(inp, title=None):
    #Imshow for Tensor
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


for idx, batch in enumerate(dataloaders["train"]):
    # Convert images (input) to float because it is a ByteTensor. Why is it a ByteTensor?
    images, labels = batch["image"].float(), batch["geohash"].float()
    out = torchvision.utils.make_grid(images)

    imshow(out)
    
"""

'\ndef imshow(inp, title=None):\n    #Imshow for Tensor\n    inp = inp.numpy().transpose((1, 2, 0))\n    mean = np.array([0.485, 0.456, 0.406])\n    std = np.array([0.229, 0.224, 0.225])\n    inp = std * inp + mean\n    inp = np.clip(inp, 0, 1)\n    plt.imshow(inp)\n    if title is not None:\n        plt.title(title)\n    plt.pause(0.001)  # pause a bit so that plots are updated\n\n\nfor idx, batch in enumerate(dataloaders["train"]):\n    # Convert images (input) to float because it is a ByteTensor. Why is it a ByteTensor?\n    images, labels = batch["image"].float(), batch["geohash"].float()\n    out = torchvision.utils.make_grid(images)\n\n    imshow(out)\n    \n'

In [13]:
t = torch.arange(10).reshape(2,5)

In [14]:
model_ft = models.resnet18(weights=ResNet18_Weights.DEFAULT)
# model_ft = models.resnet18()

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 3139)

model_ft = model_ft.to(device)



In [15]:

checkpoint = torch.load(r"F:\Users\basti\Documents\Goethe Uni\DLCV Projekt\models\pretrained.tar")
model_ft.load_state_dict(checkpoint['model_state_dict'])
optimizer_ft.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\Users\\basti\\Documents\\Goethe Uni\\DLCV Projekt\\models\\pretrained.tar'

In [ ]:
import pygeohash
def decimal_to_geohash(decimal):
    base_32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    geohash = ''
    while decimal > 0:
        geohash += base_32[decimal % 32]
        decimal //= 32
    return geohash[::-1]

def geohash_to_lat_lon(geohash):
    return pygeohash.decode(geohash)

In [ ]:
from haversine import haversine

def geogussr_score(model):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    geogussrscore = []

    # Iterate over data.
    for idx, batch in enumerate(dataloaders['val']):
        inputs, labels = batch["image"], batch["geohash"]
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        _, preds = torch.max(outputs, 1)
        _,labels = torch.max(labels, 1)


        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels)

        preds = geohashes_with_samples[preds.cpu()]
        labels = geohashes_with_samples[labels.cpu()]

        distance = []
        if len(inputs) == 1:
            distance.append(haversine(geohash_to_lat_lon(decimal_to_geohash(preds)),geohash_to_lat_lon(decimal_to_geohash(labels))))
        else:
            for i in range(len(inputs)):
                distance.append(haversine(geohash_to_lat_lon(decimal_to_geohash(preds[i])),geohash_to_lat_lon(decimal_to_geohash(labels[i]))))
           
        
        
        geogussrscore.append((5000*np.exp(-np.array(distance)/2000)).mean())
        
        

    epoch_loss = running_loss / dataset_sizes['val']
    epoch_acc = running_corrects.double() / dataset_sizes['val']
    average_score = np.array(geogussrscore).mean()
    print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Score: {average_score:.4f}' )

In [ ]:
geogussr_score(model_ft)